# Author's Note

Ultimately we want a "average patient COCI score" for each provider

This is comprised of COCI scores for each patient

    A = # visits with your attributed personnel
    B = # visits with all personnel
    COCI = A / B
    
The COCI is on a scale of 0-1 where 1 means "all visits were with my attributed provider" and 0 means "no visits were with my attributed provider"
    
To calculate this, we need a table of visits with a row for each:
* Patient (empi_id), Provider (prsnl_id), and Date (service_dt_tm)
* Optional: Pop_id, name (of provider), source_id
   
These visits come from two sources:  
* EMR (PH_F_Encounter)
* Claims (PH_F_Procedure)

# 1. Imports

In [1]:
from Ian import attribution_functions, pd, np, clusters, equal_in, plt
# import seaborn as sns; sns.set()
# import networkx as nx
# from sklearn.mixture import GaussianMixture

# %matplotlib inline

# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

# 2. Connection
Connect to Vertica with credentials and the appropriate client schema / population_id

In [2]:
# user is associate ID; password is sphere password
user = 'AO054198'
password = 'CernerSphereQ22020'
client = 'LCOX_MO'
# define a connection instance
i = attribution_functions(**{
    'host' : '10.20.71.235',
    'user' : f'{user}',
    'password' : f'{password}',
    'schema' : 'COXHEALTH',
    'population_id' : '7df18e6b-1e83-463e-9bb9-d40f40233249'
})

In [3]:
# test connection
i.query_df(f"""SELECT GETDATE()""")

,GETDATE
0,2020-03-26 14:11:58.380559


# 3. Drop Previous Tables
Drop previously created tables to make new ones

In [4]:
# query all tables related to Continuum of Care
tables = i.query_df(f"""
SELECT DISTINCT
    table_name
FROM
    tables
WHERE
    table_schema = 'DATA_INSIGHTS_ANALYST'
    AND table_name ILIKE '%continuum%'
""")

In [5]:
tables

,table_name
0,COXHEALTH_continuum_of_care_visits


In [6]:
# drop them
if tables.empty == False:
    for table in tables.table_name.unique():
        i.query_df(f"""
        DROP TABLE
            DATA_INSIGHTS_ANALYST.{table}
        """)

# 4. Make Table

[LCOX_MO Attribution Algorithm](https://wiki.cerner.com/display/LCOXHEI/Cox+Health+Attribution)

LCOX_MO Parameters
* Millennium
* OFFICE_VISIT_ENC (Outpatient Encounters)
* 12 month lookback

Other Logic
* Patients Age 18+
* Provider must be attributable
* Provider must be PCP

## 4A. Define Parameters
Look in "Custom Settings"

In [7]:
# partitions
MILLENNIUM_DATA_PARTITIONS = """dbdd2270-e366-4dcd-9ff1-e327d49d29bb
""".split('\n')
                  
data_partitions = MILLENNIUM_DATA_PARTITIONS
# visit type concepts as defined by Nate (not attribution algorithm)
visit_type_concepts = 'OFFICE_VISIT_ENC'

# look back of 12 months from date (defined by Nate as month common in literature)
visit_12_months = ('MONTH', 12)

## Extra Variables
lookback_type = visit_12_months[0]
encounter_lookback = visit_12_months[1]
procedure_lookback = visit_12_months[1]

In [8]:
# max attribution_end_date from PH_F_Person_Attribution_Outcome_Changes
date = i.query_df(i.attribution_dt_tm_query()).max_attribution_dt_tm[0]
date

Timestamp('2020-03-25 15:49:48.930000-0500', tz='tzoffset(None, -18000)')

### 4A1. Define Primary Query

In [9]:
# pop_id,codes from Ontology Table (and alias)
# given context_id, and population_id, and alias we specified
primary_concepts = f"""
		SELECT DISTINCT 
            ont.population_id,
			ont.code_oid,
			ont.code_system_id,
			ontc.alias
		FROM {i.schema}.PH_D_Ontology ont
		JOIN {i.schema}.PH_D_Ontology_Concept_Alias ontc ON (
				ont.population_id = ontc.population_id
				AND ont.concept_id = ontc.concept_id
				AND ont.context_id = ontc.context_id
				AND ont.context_id = '8FBD43EF0885489AA9FF961D66294839'
				AND ont.population_id = '{i.population_id}'
				)
		WHERE alias ='{visit_type_concepts}'
"""

## 4B. EMR / Encounters (visit_12_months query)
* Query visits between empi_id and provider_id, and dates they took place
* Specify the time range, the encounter_id, and keep within our visit_type
* Clean up poor / nonsensical data

In [10]:
# empi,provider,dates from PH_F_Encounter
# But only for Encounters with our Visit_Type (primary_concepts)
# and JOIN Enc_Prsnl_relt...
# arrival IS NOT NULL, arrived before discharge (or not discharged)
# encounters AFTER (X months before max_attribution_end_date)
# and BEFORE max_attirbution_end_date
# and encounter_id IN partitions from attribution algorithm
visit_emr = f"""
SELECT DISTINCT 
	enc.population_id,
	enc.empi_id,
	epr.provider_id,
	epr.begin_date_id,
	epr.begin_dt_tm,
	epr.end_date_id,
	epr.end_dt_tm,
	enc.service_dt_tm,
	enc.service_date_id,
	'ENCOUNTER' AS ref_record_type,
	enc.source_id
FROM {i.schema}.PH_F_Encounter enc
JOIN ({primary_concepts}) pc ON (
		enc.population_id = pc.population_id
		AND enc.encounter_type_code = pc.code_oid
		AND enc.encounter_type_coding_system_id = pc.code_system_id
		)
JOIN {i.schema}.PH_F_Encounter_Personnel_Reltn epr ON (
		enc.population_id = epr.population_id
		AND enc.encounter_id = epr.encounter_id
		AND enc.empi_id = epr.empi_id
		AND enc.actual_arrival_dt_tm IS NOT NULL
		AND (
			enc.actual_arrival_dt_tm <= enc.discharge_dt_tm
			OR enc.discharge_dt_tm IS NULL
			)
		AND (
			epr.begin_dt_tm IS NULL
			OR epr.begin_dt_tm <= enc.discharge_dt_tm
			)
		AND (
			epr.end_dt_tm IS NULL
			OR epr.end_dt_tm >= enc.actual_arrival_dt_tm
			)
		AND enc.service_dt_tm >= TIMESTAMPADD({lookback_type}, - {encounter_lookback}, '{date}')
		AND enc.service_dt_tm <= '{date}'
		AND SPLIT_PART(enc.encounter_id, ':', 1) {equal_in(data_partitions) }
		)
"""

In [11]:
# empi,provider,dates from PH_F_Encounter
# But only for Encounters with our Visit_Type (primary_concepts)
# and JOIN Enc_Prsnl_relt...
# arrival IS NOT NULL, arrived before discharge (or not discharged)
# encounters AFTER (X months before max_attribution_end_date)
# and BEFORE max_attirbution_end_date
# and encounter_id IN partitions from attribution algorithm
visit_emr = f"""
SELECT DISTINCT 
	enc.population_id,
	enc.empi_id,
	epr.provider_id,
	epr.begin_date_id,
	epr.begin_dt_tm,
	epr.end_date_id,
	epr.end_dt_tm,
	enc.service_dt_tm,
	enc.service_date_id,
	'ENCOUNTER' AS ref_record_type,
	enc.source_id
FROM {i.schema}.PH_F_Encounter enc
JOIN ({primary_concepts}) pc ON (
		enc.population_id = pc.population_id
		AND enc.encounter_type_code = pc.code_oid
		AND enc.encounter_type_coding_system_id = pc.code_system_id
		)
JOIN {i.schema}.PH_F_Encounter_Personnel_Reltn epr ON (
		enc.population_id = epr.population_id
		AND enc.encounter_id = epr.encounter_id
		AND enc.empi_id = epr.empi_id
		AND enc.actual_arrival_dt_tm IS NOT NULL
		AND (
			enc.actual_arrival_dt_tm <= enc.discharge_dt_tm
			OR enc.discharge_dt_tm IS NULL
			)
		AND (
			epr.begin_dt_tm IS NULL
			OR epr.begin_dt_tm <= enc.discharge_dt_tm
			)
		AND (
			epr.end_dt_tm IS NULL
			OR epr.end_dt_tm >= enc.actual_arrival_dt_tm
			)
		AND enc.service_dt_tm >= TIMESTAMPADD({lookback_type}, - {encounter_lookback}, '{date}')
		AND enc.service_dt_tm <= '{date}'
		AND SPLIT_PART(enc.encounter_id, ':', 1) {equal_in(data_partitions) }
		)
"""

In [12]:
# IF there is >1 row with the same empi & provider & service_date_id
# THEN keep most recent service_dt_tm
max_date_time_emr = f"""
	SELECT DISTINCT 
        ve.population_id,
		ve.empi_id,
		ve.provider_id,
		ve.service_date_id,
		MAX(ve.service_dt_tm) AS service_dt_tm
	FROM ({visit_emr}) ve
	GROUP BY ve.population_id,
		ve.empi_id,
		ve.provider_id,
		ve.service_date_id
"""

In [13]:
# Keep Visit_emr rows matching 
# this seems redundant...wouldn't you only keep max_date_time_emr
visits_12_months_max_dt_tm_query = f"""
SELECT DISTINCT ve.*
FROM ({visit_emr}) ve
JOIN ({max_date_time_emr}) mdt ON (
		ve.population_id = mdt.population_id
		AND ve.empi_id = mdt.empi_id
		AND ve.provider_id = mdt.provider_id
		AND ve.service_dt_tm = mdt.service_dt_tm
		)
"""

## 4C. Claims (visit_24_month_claims query)
* Query visits between empi_id and provider_id, and dates they took place
* Specity the time range, the encounter_id, and keep within our visit_type
* Clean up poor / nonsensical data 

In [14]:
# empi,provider,dates from PH_F_Procedure
# But only for rows with our Visit_Type (primary_concepts)
# and JOIN Proc_Prsnl_relt...
# service_start_dt_tm before service_end_dt_tm (or no end_dt_tm)
# procedure AFTER (X months before max_attribution_end_date)
# and BEFORE max_attirbution_end_date
# and procedure_id IN partitions from attribution algorithm
visit_claims = f"""
	SELECT DISTINCT 
        PROC.population_id,
		PROC.empi_id,
		ppr.provider_id,
		PROC.service_start_dt_tm,
		PROC.service_start_date_id,
		'PROCEDURE' AS ref_record_type,
		PROC.source_id
	FROM {i.schema}.PH_F_Procedure PROC
	JOIN ({primary_concepts}) pc ON (
			PROC.population_id = pc.population_id
			AND PROC.procedure_code = pc.code_oid
			AND PROC.procedure_coding_system_id = pc.code_system_id
			)
	JOIN {i.schema}.PH_F_Procedure_Personnel_Reltn ppr ON (
			PROC.population_id = ppr.population_id
			AND PROC.procedure_id = ppr.procedure_id
			AND PROC.empi_id = ppr.empi_id
            AND PROC.service_start_dt_tm IS NOT NULL
            AND (
                PROC.service_start_dt_tm <= PROC.service_end_dt_tm
                OR PROC.service_end_dt_tm IS NULL
                )
            AND (
                ppr.begin_dt_tm IS NULL
                OR ppr.begin_dt_tm <= PROC.service_end_dt_tm
                )
            AND (
                ppr.end_dt_tm IS NULL
                OR ppr.end_dt_tm >= PROC.service_start_dt_tm
                )
			AND PROC.service_start_dt_tm >= TIMESTAMPADD({lookback_type}, -{encounter_lookback}, '{date}')
			AND PROC.service_start_dt_tm <= '{date}'
			AND SPLIT_PART(SPLIT_PART(PROC.procedure_id, ':', 2), '/', 1) {equal_in(data_partitions)}
			AND PROC.claim_uid IS NOT NULL
			)
"""

In [15]:
# keep only rows where pop_id,empi,provider,start_date
max_date_time_claims = f"""
	SELECT DISTINCT 
        vc.population_id,
		vc.empi_id,
		vc.provider_id,
		vc.service_start_date_id,
		MAX(vc.service_start_dt_tm) AS service_start_dt_tm
	FROM ({visit_claims}) vc
	GROUP BY vc.population_id,
		vc.empi_id,
		vc.provider_id,
		vc.service_start_date_id
"""

In [16]:
# keep only max rows
# again...seems redundant
claims_12_months_max_dt_tm_query = f"""
SELECT DISTINCT vc.*
FROM ({visit_claims}) vc
JOIN ({max_date_time_claims}) mdt ON (
		vc.population_id = mdt.population_id
		AND vc.empi_id = mdt.empi_id
		AND vc.provider_id = mdt.provider_id
		AND vc.service_start_dt_tm = mdt.service_start_dt_tm
		)
"""

## 4D. Additional Queries

In [17]:
# the Provider table
provider_query = f"""
 SELECT DISTINCT
            provider_id,
            prsnl_id,
            name,
            given_name,
            family_name
        FROM
            {i.schema}.PH_D_Provider
"""

## 4E. Union
* Stack the EMR & Claims tables on top of eachother (union)

In [18]:
# make columns in both queries the same
# claims - service_start_dt_tm, service_start_date_id
# visits - begin_dt_tm, begin_date_id, end_dt_tm , end_date_id, service_dt_tm, service_date_id 
visits_for_union = f"""
SELECT
    population_id::VARCHAR,
    empi_id::VARCHAR,
    provider_id::VARCHAR,
    service_dt_tm,
    service_date_id::VARCHAR,
    ref_record_type::VARCHAR,
    source_id::VARCHAR
FROM ({visits_12_months_max_dt_tm_query}) v
"""
claims_for_union = f"""
SELECT
    population_id::VARCHAR,
    empi_id::VARCHAR,
    provider_id::VARCHAR,
    service_start_dt_tm as service_dt_tm,
    service_start_date_id::VARCHAR as service_date_id,
    ref_record_type::VARCHAR,
    source_id::VARCHAR
FROM ({claims_12_months_max_dt_tm_query}) c
"""

In [19]:
union_query = f"""
SELECT
    v.*
FROM ({visits_for_union}) v
UNION
SELECT
    c.*
FROM ({claims_for_union}) c
"""

## 4F. Final Filters
* Filter out patients who are NOT 18+, and patients who are deceased 

In [20]:
# All encounters from Claims & EMR
# Add prsnl_id, and provider name from PH_D_Provider
# Add birth_date, and deceased from PH_D_Person
visit_query = f"""
SELECT DISTINCT
    visit.population_id,
    prov.prsnl_id,
    prov.provider_id,
    prov.name,
    visit.empi_id,
    visit.ref_record_type,
    visit.source_id,
    visit.service_date_id,
    visit.service_dt_tm,
    p.birth_date,
    p.deceased,
    AGE_IN_YEARS(visit.service_dt_tm,p.birth_date) as age
FROM
    ({union_query}) visit
    JOIN ({provider_query}) prov ON (
        visit.provider_id = prov.provider_id
    )
    JOIN {i.schema}.PH_D_Person p ON(
        visit.empi_id = p.empi_id
        AND visit.population_id = p.population_id)
"""

In [21]:
# everything from Visit query
# Where Age is 18+ on date of encounter
# and they are not deceased
visit_query2 = f"""
SELECT DISTINCT
    v.population_id,
    v.prsnl_id,
--    v.prsnl_group_name,
    v.provider_id,
    v.name,
    v.empi_id,
    v.source_id,
    v.service_date_id,
    v.service_dt_tm,
    v.birth_date,
    v.age,
    v.deceased
FROM ({visit_query}) v
WHERE 
    v.age >=18
    AND (v.deceased !='True' OR v.deceased IS NULL)
"""

In [22]:
# Make new table from final query
i.query_df(f"""
CREATE TABLE
    DATA_INSIGHTS_ANALYST.{i.schema}_continuum_of_care_visits
AS
    {visit_query2} 
""")

""


In [23]:
# make it easier to add "WHERE" statements
coc = f"DATA_INSIGHTS_ANALYST.{i.schema}_continuum_of_care_visits"

In [24]:
# Test table
i.query_df(f"""
SELECT * FROM {coc}
""",1)

,population_id,prsnl_id,provider_id,name,empi_id,source_id,service_date_id,service_dt_tm,birth_date,age,deceased
0,7df18e6b-1e83-463e-9bb9-d40f40233249,00939109-0b08-4119-8bf8-8a5576eb0f85,4d51a50f-8f89-48af-92fc-9b43cdd1e37d,"Hull MD, Darla",c6945dd4-160e-4bbb-9161-68aeb7fd8eaf,77130780,20191002,2019-10-02 08:48:47-05:00,1963-04-23,56,None


In [25]:
# Test table
i.query_df(f"""
SELECT COUNT(DISTINCT empi_id)
FROM {coc}
""",1)

,COUNT
0,210255


In [26]:
i.query_df(f"""
SELECT COUNT(*)
FROM {coc}
""",1)

,COUNT
0,4768891


# 5. COCI 

## 5A. Describe Attribution Makeup

## 5B. Calculate the COCI (Patient Level)
One visit per day. If one of the providers was attributed, then counts towards numerator.

In [27]:
# visits per empi per provider (given they were attributed)
visit_count_per_attributed_prsnl = f"""
SELECT
    a.empi_id,
    a.prsnl_id,
    a.name,
    COUNT(DISTINCT a.service_date_id) as visit_count_attr
FROM {coc} a
    JOIN {i.schema}.PH_F_Attribution b ON(
        a.empi_id = b.empi_id
        AND a.prsnl_id = b.prsnl_id)
GROUP BY 1,2,3 ORDER BY 4 DESC
"""

# visits per empi across all providers (regardless of attribution)
visit_count_total = f"""
SELECT
    empi_id,
    COUNT(DISTINCT service_date_id) visit_count_total
FROM {coc} a
GROUP BY 1
"""

# coci =  num visits empi had with attr prsnl / num visits all prsnl
# given they had more than 1 visit_count_total
coci_query = f"""
SELECT DISTINCT
    a.empi_id,
    a.prsnl_id,
    a.name,
    a.visit_count_attr,
    b.visit_count_total,
    CAST(ROUND(a.visit_count_attr / b.visit_count_total,3) AS FLOAT) as COCI
FROM ({visit_count_per_attributed_prsnl}) a
    JOIN ({visit_count_total}) b ON(
        a.empi_id = b.empi_id
        AND b.visit_count_total>1)
"""
coci_df = i.query_df(coci_query)

In [28]:
coci_df.prsnl_id.nunique()

466

In [29]:
coci_df.head(1)

,empi_id,prsnl_id,name,visit_count_attr,visit_count_total,COCI
0,ac74c86a-b065-4a73-bf9c-dd57e605e73f,447b71e2-97c1-45ac-a05d-99b0001a7714,"Mabe MD, Jeremy D",3,3,1.0


# 5B(1). Calculate COCI (Patient Level) | Attributable prsnl

In [30]:
# visits per empi per provider (given they were attributed)
visit_count_per_attributed_prsnl = f"""
SELECT
    a.empi_id,
    a.prsnl_id,
    a.name,
    COUNT(DISTINCT a.service_date_id) as visit_count_attr
FROM {coc} a
    JOIN {i.schema}.PH_F_Attribution b ON(
        a.empi_id = b.empi_id
        AND a.prsnl_id = b.prsnl_id)
GROUP BY 1,2,3 ORDER BY 4 DESC
"""

# visits per empi across all providers (given they were attributable)
visit_count_total = f"""
SELECT
    a.empi_id,
    COUNT(DISTINCT a.service_date_id) visit_count_total
FROM {coc} a
JOIN {i.schema}.PH_F_Attribution b ON(
    a.prsnl_id = b.prsnl_id)
GROUP BY 1
"""

# coci =  num visits empi had with attr prsnl / num visits all attributable prsnl
# given they had more than 1 visit_count_total
coci_query = f"""
SELECT DISTINCT
    a.empi_id,
    a.prsnl_id,
    a.name,
    a.visit_count_attr,
    b.visit_count_total as visit_count_total_attr,
    CAST(ROUND(a.visit_count_attr / b.visit_count_total,3) AS FLOAT) as COCI
FROM ({visit_count_per_attributed_prsnl}) a
    JOIN ({visit_count_total}) b ON(
        a.empi_id = b.empi_id
        AND b.visit_count_total>1)
"""
coci_df_attr = i.query_df(coci_query)

In [31]:
coci_df.prsnl_id.nunique()

466

In [32]:
coci_df_attr.prsnl_id.nunique()

465

In [33]:
coci_df.head(1)

,empi_id,prsnl_id,name,visit_count_attr,visit_count_total,COCI
0,ac74c86a-b065-4a73-bf9c-dd57e605e73f,447b71e2-97c1-45ac-a05d-99b0001a7714,"Mabe MD, Jeremy D",3,3,1.0


In [34]:
coci_merged_df = coci_df_attr.merge(coci_df,how='outer')

In [35]:
coci_merged_df.head(1)

,empi_id,prsnl_id,name,visit_count_attr,visit_count_total_attr,COCI,visit_count_total
0,7e63d7a6-0a0f-4571-87d6-da2261093ec0,8ce5e04b-3e3e-4db1-ba81-04693a5e65ca,"Jett MD, Dayna Rae",3,8.0,0.375,NaN


## 6. Calculate the COCI (Prsnl Level)

In [36]:
# empi should already be filtered to only empi who had 2+ visits
# keep visits only if the prsnl_id was in the Attribution table
attributable_prsnl = f"""
SELECT
    a.empi_id,
    a.prsnl_id,
    a.name,
    a.COCI
FROM ({coci_query}) a
JOIN {i.schema}.PH_F_Attribution b ON(
    a.prsnl_id = b.prsnl_id)
"""

# for each prsnl, average coci, count empi
prov_rollup = f"""
SELECT
a.prsnl_id,
a.name,
--a.prsnl_group_name,
COUNT(DISTINCT a.empi_id) as coci_empi_ct,
CAST(ROUND(AVG(a.COCI),3) AS FLOAT) as avg_coci
FROM ({attributable_prsnl}) a
GROUP BY 1,2
"""
# empi_id attributed to each prov_id
attr_coci = f"""
SELECT
a.prsnl_id,
COUNT(DISTINCT empi_id) attr_empi_ct
FROM {i.schema}.PH_F_Attribution a
GROUP BY 1
"""
# combine the tables
coci_summary = f"""
SELECT
a.prsnl_id,
a.name,
a.avg_coci,
a.coci_empi_ct,
b.attr_empi_ct
FROM ({prov_rollup}) a
JOIN ({attr_coci}) b ON(
    a.prsnl_id = b.prsnl_id)
"""
coci_prsnl_df = i.query_df(coci_summary)

In [37]:
coci_prsnl_df.prsnl_id.nunique()

465

In [38]:
coci_prsnl_df.head(1)

,prsnl_id,name,avg_coci,coci_empi_ct,attr_empi_ct
0,0b005c8e-b62d-445b-bc27-53d64ef7b302,"Albrecht FNP, Faye",0.427,107,343


In [39]:
# # export patient-level or provider-level continuity data
# coci_prsnl_df.to_csv(f"{i.schema}_coci_prsnl_df.csv")
# coci_df.to_csv(f"{i.schema}_coci_empi_df.csv")

# 7. Investigation / Validation

In [40]:
# # choose a prsnl
# coci_prsnl_df.sample(1)

In [41]:
# prsnl_to_investigate = "1b3c9bfd-b8b1-4471-b130-6483cc853172"

### 7.1 Prsnl COCI

In [42]:
# # what were the visits / COCI for the provider?
# coci_prsnl_df.loc[coci_prsnl_df.prsnl_id==prsnl_to_investigate]

In [43]:
# # what were the visits / COCI for the provider?
# coci_prsnl_df.loc[coci_prsnl_df.prsnl_id==prsnl_to_investigate]

### 7.2 Empi_id COCI

In [44]:
# # what were the visits / COCI for their patients?
# coci_df.loc[coci_df.prsnl_id==prsnl_to_investigate].reset_index()

In [45]:
# # what were the visits / COCI for their patients?
# coci_df.loc[coci_df.prsnl_id==prsnl_to_investigate].reset_index()

### 7.3 Empi_id Visits

In [46]:
# # pick an empi_id to investigate
# empi_to_investigate = '5bc7338b-6f0f-4076-a6f7-5842b35c5917'

In [47]:
# # empi_id numerator
# i.query_df(f"""
# SELECT
#     empi_id,
#     COUNT(DISTINCT service_date_id) visit_count_total
# FROM ({coc_mmary})
# WHERE empi_id = '{empi_to_investigate}'
# GROUP BY 1
# """,1)

In [48]:
# # emp_id visits per provider
# i.query_df(f"""
# SELECT DISTINCT
# a.name,
# COUNT(DISTINCT a.service_date_id)
# FROM {coc} a
# WHERE empi_id = '{empi_to_investigate}'
# GROUP BY 1
# ORDER BY a.name
# """)

# 8. Organization

In [49]:
coci_prsnl_df.head(1)

,prsnl_id,name,avg_coci,coci_empi_ct,attr_empi_ct
0,0b005c8e-b62d-445b-bc27-53d64ef7b302,"Albrecht FNP, Faye",0.427,107,343


In [50]:
org_df = i.query_df(f"""
SELECT DISTINCT
	oc.org_class_name,
	org.org_name,
	prsnl.prsnl_id,
	INITCAP(prsnl.prsnl_name) AS prsnl_name,
	MAX(CASE WHEN pg.prsnl_group_name ILIKE '%scorable%' THEN 1 ELSE 0 END) AS scorable_ind,
	MAX(CASE WHEN pg.prsnl_group_name ILIKE '%attributable%' THEN 1 ELSE 0 END) AS attributable_ind,
	MAX(CASE WHEN pg.prsnl_group_mnemonic ILIKE '%scoring%' THEN 1 ELSE 0 END) AS scoring_personnel_group_mnemonic,
        scoring_group.prsnl_group_name AS scoring_group,
	sc_prsnl.scoring_group AS scorecard_scoring_group_id,
	sp.name AS scoring_plan_name,
	opr.reltn_type AS organization_relationship_type,
     COUNT(DISTINCT(attr.empi_id)) AS attributable_empi_count
FROM
	{i.schema}.PH_D_Organization org
	LEFT OUTER JOIN {i.schema}.PH_D_Organization_Organization_Class_Reltn oocr ON (
		org.org_id = oocr.org_id
		AND org.version = oocr.version
	)
	LEFT OUTER JOIN {i.schema}.PH_D_Organization_Class oc ON (
		oc.org_class_id = oocr.org_class_id 
		AND oc.version = oocr.version
	)
	LEFT OUTER JOIN {i.schema}.PH_D_Organization_Personnel_Reltn opr ON 
	(
		org.org_id = opr.org_id
		AND org.version = opr.version
	)
	LEFT OUTER JOIN {i.schema}.PH_D_Personnel prsnl ON (
		opr.prsnl_id = prsnl.prsnl_id
		AND opr.version = prsnl.version
	)
	LEFT OUTER JOIN {i.schema}.PH_F_Scorecard_Personnel sc_prsnl ON (
		prsnl.prsnl_id = sc_prsnl.prsnl_id
                AND prsnl.version = sc_prsnl.version
	)
	LEFT OUTER JOIN {i.schema}.PH_D_Scoring_Plan sp ON (
		sc_prsnl.scoring_plan_id = sp.scoring_plan_id
                AND sc_prsnl.version = sp.version
	)
	LEFT OUTER JOIN {i.schema}.PH_D_Personnel_Personnel_Group_Reltn ppgr ON (
		ppgr.prsnl_id = prsnl.prsnl_id
                AND ppgr.version = prsnl.version
	)
	LEFT OUTER JOIN {i.schema}.PH_D_Personnel_Group pg ON (
		pg.prsnl_group_id = ppgr.prsnl_group_id
                AND pg.version = ppgr.version
	)
	LEFT OUTER JOIN (
		SELECT
			ppgr.prsnl_id,
			pg.prsnl_group_name
		FROM
			{i.schema}.PH_D_Personnel_Personnel_Group_Reltn ppgr
			INNER JOIN {i.schema}.PH_D_Personnel_Group pg ON (
				ppgr.prsnl_group_id = pg.prsnl_group_id
			)
		WHERE
			pg.prsnl_group_mnemonic = 'scoring'
	) scoring_group ON (
		scoring_group.prsnl_id = prsnl.prsnl_id
	)
	LEFT OUTER JOIN {i.schema}.PH_F_Attribution attr ON 
	(
		attr.prsnl_id = prsnl.prsnl_id
	)
WHERE
	prsnl.active_ind = TRUE
GROUP BY
	1,2,3,4,8,9,10,11
ORDER BY
	org_class_name,
	org_name,
	prsnl_name
""")

In [51]:
org_df.head(1)

,org_class_name,org_name,prsnl_id,prsnl_name,scorable_ind,attributable_ind,scoring_personnel_group_mnemonic,scoring_group,scorecard_scoring_group_id,scoring_plan_name,organization_relationship_type,attributable_empi_count
0,CMG,Adult Medicine & Endocrinology Specialists,7b0bb72b-abfd-4c4c-9550-eeb6f9672bea,"Allen Fnp, Helen M",0,1,1,NPP PCP,None,None,MEMBER,2


In [52]:
coci_org = coci_prsnl_df.merge(org_df,how='left')

In [53]:
coci_org.head(1)

,prsnl_id,name,avg_coci,coci_empi_ct,attr_empi_ct,org_class_name,org_name,prsnl_name,scorable_ind,attributable_ind,scoring_personnel_group_mnemonic,scoring_group,scorecard_scoring_group_id,scoring_plan_name,organization_relationship_type,attributable_empi_count
0,0b005c8e-b62d-445b-bc27-53d64ef7b302,"Albrecht FNP, Faye",0.427,107,343,CMG,Cox Senior Health Center South,"Albrecht Fnp, Faye",1.0,1.0,1.0,NPP PCP,None,None,MEMBER,343.0


In [54]:
coci_org.to_csv(f"coci_org_{i.schema}_attr.csv")